In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore")
import random



In [4]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set your desired seed
set_seed(100)

In [ ]:
#Notes Regarding versions:
#3.6.13 python --> 3.8.10
#0.10.0 gluonts --> 0.9.0
#Version: 1.1.5 pandas --> 1.5.3
#Version: 1.8.0 torch --> 2.1.5 --> 1.10.0 last version according to the issue answer in git
#Version: 1.19.5 numpy --> 1.23.5
#after this had issue regarding the distribution output importing changed the importing in the pts/dis_output line 34 to gluonts.torch.modules.distribution_output
# and remove it from the importing cell.
#encountered prefetch error handled it by downgrading torch version
#to install gluonts 0.9.0 had to downgrade pip version to 24.0
#!pip install optuna

In [3]:
import numpy as np
import pandas as pd
import torch
import sys
import pts.dataset
#sys.path.append(r'C:\Users\usama\Desktop\Thesis\pytorch-ts-master')

In [5]:
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
#import gluonts.torch.distributions.distribution_output
from gluonts.dataset.repository.datasets import dataset_recipes, get_dataset
from pts.model.tempflow import TempFlowEstimator
from pts.model.transformer_tempflow import TransformerTempFlowEstimator
from pts import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import MultivariateEvaluator

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
dataset = get_dataset("solar_nips", regenerate=False)
dataset.metadata

MetaData(freq='H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat_0', cardinality='137')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=24)

In [8]:
# #for electricity data
# from gluonts.dataset.common import ListDataset

# # Define train and validation datasets separately
# train_data = []
# val_data = []

# for i, entry in enumerate(dataset.train):
#     series_length = len(entry['target'])
#     train_size = int(0.8 * series_length)  # 80% of the actual series length
#     val_size = series_length - train_size  # Remaining 20%
    
#     # Create train entry
#     train_entry = entry.copy()
#     train_entry['target'] = entry['target'][:train_size]  # First 80%
#     train_data.append(train_entry)
    
#     # Create validation entry
#     val_entry = entry.copy()
#     val_entry['target'] = entry['target'][train_size:]  # Remaining 20%
#     val_data.append(val_entry)
    
#     # Print verification
#     print(f"Series {i}: Train length = {train_size}, Validation length = {val_size}")
    
#     # Assert the lengths are correct
#     assert len(train_entry['target']) == train_size, f"Train length mismatch for series {i}"
#     assert len(val_entry['target']) == val_size, f"Validation length mismatch for series {i}"


In [9]:
train_grouper = MultivariateGrouper(max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))
test_grouper = MultivariateGrouper(num_test_dates=int(len(dataset.test)/len(dataset.train)), 
                                    max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))
# val_grouper = MultivariateGrouper(max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))
# test_grouper = MultivariateGrouper(num_test_dates=int(len(dataset.test)/len(train_data)), 
#                                    max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))

In [10]:
# train_dataset = ListDataset(train_data, freq=dataset.metadata.freq)
# validation_dataset = ListDataset(val_data, freq=dataset.metadata.freq)

In [11]:
#dataset_train = train_grouper(train_dataset)
dataset_train = train_grouper(dataset.train)
dataset_test = test_grouper(dataset.test)
# dataset_val = val_grouper(validation_dataset)

In [12]:
evaluator = MultivariateEvaluator(quantiles=(np.arange(20)/20.0)[1:],
                                  target_agg_funcs={'sum': np.sum})

In [14]:
import numpy as np
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import MultivariateEvaluator

# Define the grid for hyperparameter tuning
learning_rates = [1e-5, 1e-4, 1e-3, 1e-2]  # Example learning rates
n_blocks_list = [2, 3, 4, 5, 6]  # Number of blocks to try

# Initialize variables to store the best results
best_metric = float('inf')  # Smallest CRPS-Sum
best_params = None

# Multivariate evaluator for evaluation
evaluator = MultivariateEvaluator(
    quantiles=(np.arange(20) / 20.0)[1:], target_agg_funcs={'sum': np.sum}
)

# Iterate over all combinations of hyperparameters
for learning_rate in learning_rates:
    for n_blocks in n_blocks_list:
        print(f"Trying learning_rate={learning_rate}, n_blocks={n_blocks}")
        
        # Define the estimator
        estimator = TempFlowEstimator(
            target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
            prediction_length=24,
            cell_type='GRU',
            input_size=552,
            freq=dataset.metadata.freq,
            scaling=True,
            dequantize=True,
            n_blocks=n_blocks,
            trainer=Trainer(
                device=device,
                epochs=1,
                learning_rate=learning_rate,
                num_batches_per_epoch=100,
                batch_size=64,
            ),
        )
        
        # Train and evaluate the model
        predictor = estimator.train(dataset_train, num_workers=4)
        forecast_it, ts_it = make_evaluation_predictions(
            dataset=dataset_test, predictor=predictor, num_samples=100
        )
        forecasts = list(forecast_it)
        targets = list(ts_it)
        
        # Evaluate the forecasts
        agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))
        current_metric = agg_metric['m_sum_mean_wQuantileLoss']
        
        print(f"CRPS-Sum for learning_rate={learning_rate}, n_blocks={n_blocks}: {current_metric}")
        
        # Update the best parameters if the current metric is better
        if current_metric < best_metric:
            best_metric = current_metric
            best_params = {
                'learning_rate': learning_rate,
                'n_blocks': n_blocks,
            }

# Print the best results
print(f"Best CRPS-Sum: {best_metric}")
print(f"Best Hyperparameters: {best_params}")


Trying learning_rate=1e-05, n_blocks=2


AttributeError: module 'torch.nn' has no attribute 'linear'

In [ ]:
estimator = TempFlowEstimator(
    target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
    prediction_length=dataset.metadata.prediction_length,
    cell_type='GRU',
    input_size=1484,
    freq=dataset.metadata.freq,
    scaling=True,
    dequantize=True,
    n_blocks=4,
    trainer=Trainer(device=device,
                    epochs=45,
                    learning_rate=0.013074418981914738,
                    num_batches_per_epoch=100,
                    batch_size=64)
)

In [ ]:
predictor = estimator.train(dataset_train, num_workers =4)
forecast_it, ts_it = make_evaluation_predictions(dataset=dataset_test,
                                             predictor=predictor,
                                             num_samples=100)
forecasts = list(forecast_it)
targets = list(ts_it)

agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))

In [ ]:
nlls = []
for target, forecast in zip(targets, forecasts):
    # Convert target to a numpy array if it is not already
    target_array = np.asarray(target)  # Ensure the target is a numpy array

    # Extract the target values for the prediction length
    observed = target_array[-predictor.prediction_length:]  # Shape: (prediction_length,)

    # Get forecast samples (shape: [num_samples, prediction_length, target_dim])
    samples = forecast.samples  # Shape: (100, 24, 370)

    # If multivariate, ensure dimensions match
    if samples.ndim == 3:  # (num_samples, prediction_length, target_dim)
        for dim in range(samples.shape[-1]):  # Iterate over dimensions
            dim_samples = samples[:, :, dim]  # Shape: (100, 24)
            dim_observed = observed[:, dim]   # Extract observed values for this dimension

            # Estimate likelihoods for observed values under the empirical distribution
            likelihoods = np.mean(
                np.exp(-0.5 * ((dim_observed - dim_samples) ** 2)), axis=0
            )  # Shape: (24,)
            log_likelihoods = np.log(likelihoods + 1e-10)  # Add epsilon for numerical stability

            # Compute NLL for this dimension
            nll = -np.mean(log_likelihoods)
            nlls.append(nll)
    else:  # For univariate cases
        likelihoods = np.mean(
            np.exp(-0.5 * ((observed - samples) ** 2)), axis=0
        )  # Shape: (24,)
        log_likelihoods = np.log(likelihoods + 1e-10)  # Add epsilon for numerical stability

        # Compute NLL for univariate case
        nll = -np.mean(log_likelihoods)
        nlls.append(nll)

# Aggregate NLL across all series and dimensions
average_nll = np.mean(nlls)

# Print the result
print(f"Negative Log-Likelihood (NLL): {average_nll}")

In [ ]:
print("CRPS: {}".format(agg_metric['mean_wQuantileLoss']))
print("ND: {}".format(agg_metric['ND']))
print("NRMSE: {}".format(agg_metric['NRMSE']))
print("MSE: {}".format(agg_metric['MSE']))

In [ ]:
print("CRPS-Sum: {}".format(agg_metric['m_sum_mean_wQuantileLoss']))
print("ND-Sum: {}".format(agg_metric['m_sum_ND']))
print("NRMSE-Sum: {}".format(agg_metric['m_sum_NRMSE']))
print("MSE-Sum: {}".format(agg_metric['m_sum_MSE']))

### Tunning for Electricity

In [ ]:
import optuna
import torch

# Objective function for Optuna
def objective(trial):
    # Suggest values for hyperparameters
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-7, 1e-1)
    #batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    #epochs = trial.suggest_int("epochs", 25, 75)
    n_blocks = trial.suggest_int("n_blocks", 2, 8)
    
    # Initialize the Trainer with suggested parameters
    trainer = Trainer(
        device=device,
        epochs=10,
        learning_rate=learning_rate,
        batch_size=64,
        num_batches_per_epoch=100
    )
    
    # Initialize the Estimator with suggested parameters
    estimator = TempFlowEstimator(
        target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
        prediction_length=dataset.metadata.prediction_length,
        cell_type='GRU',
        input_size=1484,
        freq=dataset.metadata.freq,
        scaling=True,
        dequantize=True,
        n_blocks=n_blocks,
        trainer=trainer
    )
    
    # Train the model and get the predictor
    predictor = estimator.train(dataset_train, dataset_val, num_workers=4)
    
    # Return the validation loss for the current trial (assuming Trainer has avg_val_loss attribute)
    return trainer.avg_val_loss

# Run the Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1)

# Output best parameters and validation loss
print("Best parameters found:", study.best_params)
print("Best validation loss:", study.best_value)
